## 第一步：安装依赖

In [3]:
# 安装 unsloth 包。unsloth 是一个用于微调大型语言模型（LLM）的工具，可以让模型运行更快、占用更少内存。
# 卸载当前已安装的 unsloth 包（如果已安装），然后从 GitHub 的源代码安装最新版本。
# 这样可以确保我们使用的是最新功能和修复。
!pip uninstall unsloth -y
!pip install unsloth

!pip install bitsandbytes unsloth_zoo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.7/192.7 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 96.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 第二步：加载预训练模型

In [4]:
from unsloth import FastLanguageModel  # 导入FastLanguageModel类，用来加载和使用模型
import torch  # 导入torch工具，用于处理模型的数学运算

max_seq_length = 2048  # 设置模型处理文本的最大长度，相当于给模型设置一个“最大容量”
dtype = None  # 设置数据类型，让模型自动选择最适合的精度
load_in_4bit = True  # 使用4位量化来节省内存，就像把大箱子压缩成小箱子

# 加载预训练模型，并获取tokenizer工具
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",  # 指定要加载的模型名称
    max_seq_length=max_seq_length,  # 使用前面设置的最大长度
    dtype=dtype,  # 使用前面设置的数据类型
    load_in_4bit=load_in_4bit,  # 使用4位量化
    # token="hf_...",  # 如果需要访问授权模型，可以在这里填入密钥
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.51.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/53.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

## 第三步：微调前测试

In [5]:
prompt_style = """以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是一位在临床推理，诊断和治疗方面计划方面具有高级知识的医学专家。
请回答以下医疗问题。

### 问题：
{}

### 回答：
<think>{}"""
# 定义提示风格的字符串模板，用于格式化问题

question = "根据患者59岁女性，出现胸闷、憋喘症状和相关的体格检查结果，其中X线显示双侧胸腔积液，胸水检查结果显示血性积液且有异常细胞，痰细胞学检查提示癌细胞体积小、圆形或卵圆形、胞质少，核畸形并深染，结合这些检查结果，患者最可能的诊断是什么？"
# 定义具体的医学问题

In [6]:
FastLanguageModel.for_inference(model)
# 准备模型以进行推理

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")
# 使用 tokenizer 对格式化后的问题进行编码，并移动到 GPU

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)
# 使用模型生成回答

response = tokenizer.batch_decode(outputs)
# 解码模型生成的输出为可读文本

print(response[0])
# 打印生成的回答部分

<｜begin▁of▁sentence｜>以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是一位在临床推理，诊断和治疗方面计划方面具有高级知识的医学专家。
请回答以下医疗问题。

### 问题：
根据患者59岁女性，出现胸闷、憋喘症状和相关的体格检查结果，其中X线显示双侧胸腔积液，胸水检查结果显示血性积液且有异常细胞，痰细胞学检查提示癌细胞体积小、圆形或卵圆形、胞质少，核畸形并深染，结合这些检查结果，患者最可能的诊断是什么？

### 回答：
<think>
好的，我现在要分析这位59岁女性的症状和检查结果，来确定她最可能的诊断。首先，患者主要症状是胸闷和憋喘，这通常提示呼吸系统的问题，可能涉及肺部疾病。接下来，体格检查显示双侧胸腔积液，X线结果也证实了这一点。胸水检查显示血性积液且有异常细胞，这可能提示积液中存在血细胞或其他异常细胞。

再看痰细胞学检查，结果显示癌细胞体积小、圆形或卵圆形、胞质少，核畸形并深染。这表明痰液中存在癌细胞，可能来自于肺癌或其他原发性癌。结合这些信息，我需要考虑患者的年龄、症状和检查结果之间的关联。

肺癌在中老年患者中较为常见，尤其是吸烟史较多的患者。而患者的症状如胸闷和憋喘，尤其是伴随积液，可能提示肺癌。另外，血性积液和异常细胞可能暗示了肺脓肿或肺栓塞，但肺癌也是其中的常见原因。

考虑到患者的年龄和症状，肺癌是最可能的诊断。因此，我认为患者最可能的诊断是肺癌，具体类型可能需要进一步的检查来确定。
</think>

根据患者59岁的年龄、胸闷和憋喘症状，以及X线和胸水检查显示的双侧胸腔积液和异常细胞，结合痰细胞学检查结果显示的癌细胞特征，最可能的诊断是肺癌。肺癌在中老年患者中较为常见，尤其是吸烟史较多的患者。胸闷和憋喘可能由肺癌引起，积液和异常细胞进一步支持了这一诊断。因此，患者最可能的诊断是肺癌。<｜end▁of▁sentence｜>


## 第四步：加载数据集

In [7]:
# 定义一个用于格式化提示的多行字符串模板
train_prompt_style = """以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是一位在临床推理，诊断和治疗方面计划方面具有高级知识的医学专家。
请回答以下医疗问题。

### 问题：
{}

### 回答：
<思考>
{}
</思考>
{}"""

In [8]:
# 定义结束标记（EOS_TOKEN），用于指示文本的结束
EOS_TOKEN = tokenizer.eos_token  # 必须添加结束标记

from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("FreedomIntelligence/Medical-R1-Distill-Data-Chinese")
print(ds.column_names)

README.md:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

medical_r1_distill_sft_Chinese.json:   0%|          | 0.00/124M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/17000 [00:00<?, ? examples/s]

{'train': ['question', 'reasoning (reasoning_content)', 'response (content)']}


In [9]:
# 定义一个函数，用于格式化数据集中的每条记录
def formatting_prompts_func(examples):
    # 从数据集中提取问题、复杂思考过程和回答
    inputs = examples["question"]
    cots = examples["reasoning (reasoning_content)"]
    outputs = examples["response (content)"]
    texts = []  # 用于存储格式化后的文本
    # 遍历每个问题、思考过程和回答，进行格式化
    for input, cot, output in zip(inputs, cots, outputs):
        # 使用字符串模板插入数据，并加上结束标记
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)  # 将格式化后的文本添加到列表中
    return {
        "text": texts,  # 返回包含所有格式化文本的字典
    }
for split in ds:
    ds[split] = ds[split].map(formatting_prompts_func, batched=True)
print(ds["train"]["text"][0])  # 查看训练集的第一个样本

Map:   0%|          | 0/17000 [00:00<?, ? examples/s]

以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是一位在临床推理，诊断和治疗方面计划方面具有高级知识的医学专家。
请回答以下医疗问题。

### 问题：
根据患者59岁女性，出现胸闷、憋喘症状和相关的体格检查结果，其中X线显示双侧胸腔积液，胸水检查结果显示血性积液且有异常细胞，痰细胞学检查提示癌细胞体积小、圆形或卵圆形、胞质少，核畸形并深染，结合这些检查结果，患者最可能的诊断是什么？

### 回答：
<思考>
嗯，好的，我现在要解决这个病例的问题。首先，患者是59岁女性，主诉是胸闷和憋喘。体格检查发现有双侧胸腔积液，X线确认了这一点。胸水检查显示是血性积液，并且有异常细胞。痰细胞学检查发现癌细胞体积小，呈圆形或卵圆形，胞质少，核畸形且深染。我需要根据这些信息来推断最可能的诊断。

首先，我应该回顾一下胸腔积液的常见原因。胸腔积液可能由多种原因引起，包括充血性心力衰竭、感染（如肺炎旁积液）、恶性肿瘤、肺栓塞、结核等。但这里胸水是血性的，并且发现了异常细胞，所以恶性肿瘤的可能性更大。血性胸腔积液常见于恶性肿瘤，尤其是肺癌转移至胸膜，或者间皮瘤，或者其他转移癌，比如乳腺癌、卵巢癌等转移到胸膜。但患者是女性，59岁，需要考虑原发肺癌的可能。

接下来，痰细胞学检查的结果提示癌细胞体积小、圆形或卵圆形，胞质少，核畸形深染。这些特征可能指向某种特定类型的肺癌。肺癌的主要类型有小细胞癌、鳞癌、腺癌和大细胞癌。其中，小细胞肺癌的细胞学特征通常是细胞小，呈圆形或卵圆形，胞质少，核染色深，核畸形可能不太明显，但可能有挤压现象。而腺癌的细胞通常较大，胞质可能较多，形成腺泡状结构，核可能较大，但这里的描述是体积小，所以可能更倾向于小细胞癌。

不过，小细胞肺癌通常发生在中央型，容易早期转移，常见于吸烟者。患者是否有吸烟史呢？但题目中没有提到，所以不能确定。但痰中发现癌细胞，说明肿瘤可能位于支气管内，容易脱落到痰中，这可能更符合中央型肺癌，如小细胞癌或鳞癌。但鳞癌的细胞学特征通常是较大的细胞，胞质较多，角化现象，核深染但可能有更明显的核仁，而这里的描述是细胞小，所以可能更符合小细胞癌。

另外，双侧胸腔积液，胸水中有异常细胞，可能提示胸膜转移，也就是恶性胸

## 第五步：执行微调

In [10]:
FastLanguageModel.for_training(model)

model = FastLanguageModel.get_peft_model(
    model,  # 传入已经加载好的预训练模型
    r = 16,  # 设置 LoRA 的秩，决定添加的可训练参数数量
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",  # 指定模型中需要微调的关键模块
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,  # 设置 LoRA 的超参数，影响可训练参数的训练方式
    lora_dropout = 0,  # 设置防止过拟合的参数，这里设置为 0 表示不丢弃任何参数
    bias = "none",    # 设置是否添加偏置项，这里设置为 "none" 表示不添加
    use_gradient_checkpointing = "unsloth",  # 使用优化技术节省显存并支持更大的批量大小
    random_state = 3407,  # 设置随机种子，确保每次运行代码时模型的初始化方式相同
    use_rslora = False,  # 设置是否使用 Rank Stabilized LoRA 技术，这里设置为 False 表示不使用
    loftq_config = None,  # 设置是否使用 LoftQ 技术，这里设置为 None 表示不使用
)

Unsloth 2025.3.19 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [11]:
from trl import SFTTrainer  # 导入 SFTTrainer，用于监督式微调
from transformers import TrainingArguments  # 导入 TrainingArguments，用于设置训练参数
from unsloth import is_bfloat16_supported  # 导入函数，检查是否支持 bfloat16 数据格式

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=ds['train'],  # Use 'ds' instead of 'dataset' and select the 'train' split
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=75,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/17000 [00:00<?, ? examples/s]

In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 17,000 | Num Epochs = 1 | Total steps = 75
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.512100
2,1.493100
3,1.364200
4,1.353600
5,1.333800
6,1.337400
7,1.197500
8,1.212100
9,1.252500
10,1.203500


Step,Training Loss
1,1.512100
2,1.493100
3,1.364200
4,1.353600
5,1.333800
6,1.337400
7,1.197500
8,1.212100
9,1.252500
10,1.203500


## 第七步：微调后测试

In [13]:
print(question) # 打印前面的问题

根据患者59岁女性，出现胸闷、憋喘症状和相关的体格检查结果，其中X线显示双侧胸腔积液，胸水检查结果显示血性积液且有异常细胞，痰细胞学检查提示癌细胞体积小、圆形或卵圆形、胞质少，核畸形并深染，结合这些检查结果，患者最可能的诊断是什么？


In [14]:
# 将模型切换到推理模式，准备回答问题
FastLanguageModel.for_inference(model)

# 将问题转换成模型能理解的格式，并发送到 GPU 上
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

# 让模型根据问题生成回答，最多生成 4000 个新词
outputs = model.generate(
    input_ids=inputs.input_ids,  # 输入的数字序列
    attention_mask=inputs.attention_mask,  # 注意力遮罩，帮助模型理解哪些部分重要
    max_new_tokens=4000,  # 最多生成 4000 个新词
    use_cache=True,  # 使用缓存加速生成
)

# 将生成的回答从数字转换回文字
response = tokenizer.batch_decode(outputs)

# 打印回答
print(response[0])

<｜begin▁of▁sentence｜>以下是描述任务的指令，以及提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请仔细思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令：
你是一位在临床推理，诊断和治疗方面计划方面具有高级知识的医学专家。
请回答以下医疗问题。

### 问题：
根据患者59岁女性，出现胸闷、憋喘症状和相关的体格检查结果，其中X线显示双侧胸腔积液，胸水检查结果显示血性积液且有异常细胞，痰细胞学检查提示癌细胞体积小、圆形或卵圆形、胞质少，核畸形并深染，结合这些检查结果，患者最可能的诊断是什么？

### 回答：
<think>
根据患者59岁女性的临床表现、体格检查和实验室检查结果，结合X线显示双侧胸腔积液、胸水检查为血性积液且有异常细胞，以及痰细胞学检查提示癌细胞特征，最可能的诊断是**胸腺癌转移性肺结节**。

### 分析步骤：
1. **病史与症状**：患者出现胸闷、憋喘症状，结合胸腔积液，提示可能存在胸腺癌转移至肺部的肿瘤。
2. **X线和胸水检查**：双侧胸腔积液提示肺部有转移病灶，胸水检查显示血性积液（可能含有血液或血管内肿瘤），并伴有异常细胞（转移性肿瘤细胞）。
3. **痰细胞学检查**：癌细胞体积小、圆形或卵圆形，胞质少、核畸形且深染，符合转移性肿瘤细胞的特征。
4. **临床推断**：胸腺癌（如甲状腺癌）常见于年轻女性，转移至肺部的可能性较高。胸腔积液和肺结节是常见的转移部位，结合实验室结果支持肿瘤细胞存在。

### 建议治疗：
1. **影像学检查**：进一步CT或MRI评估肺部结节大小和位置，评估是否有淋巴结转移。
2. **肿瘤标记物检测**：如甲状腺癌相关标记物（如CEA、TgAb）可能帮助明确病因。
3. **治疗方案**：根据肿瘤大小和程度选择手术、放疗或靶向治疗（如抗PD-L1治疗），同时考虑肺功能评估和心理支持。

此诊断需结合全身检查确保无其他转移部位，确保治疗方案全面。<｜end▁of▁sentence｜>


## 第八步：将微调后的模型保存为 GGUF 格式



In [18]:
# 导入 Google Colab 的 userdata 模块，用于访问用户数据
from google.colab import userdata

# 从 Google Colab 用户数据中获取 Hugging Face 的 API 令牌
HUGGINGFACE_TOKEN = userdata.get('专业团队')

# 将模型保存为 8 位量化格式（Q8_0）
# 这种格式文件小且运行快，适合部署到资源受限的设备
if True: model.save_pretrained_gguf("model", tokenizer,)

# 将模型保存为 16 位量化格式（f16）
# 16 位量化精度更高，但文件稍大
if False: model.save_pretrained_gguf("model_f16", tokenizer, quantization_method = "f16")

# 将模型保存为 4 位量化格式（q4_k_m）
# 4 位量化文件最小，但精度可能稍低
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.
Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.0G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.63 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 47%|████▋     | 15/32 [00:01<00:01, 12.73it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [01:50<00:00,  3.45s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00004.bin...
Unsloth: Saving model/pytorch_model-00002-of-00004.bin...
Unsloth: Saving model/pytorch_model-00003-of-00004.bin...
Unsloth: Saving model/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at model into q8_0 GGUF format.
The output location will be /content/model/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F32, shape = {64}
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00004.bin

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: /content/model/unsloth.Q8_0.gguf


### 第九步：将微调后的模型上传到 HuggingFace


In [19]:
# 导入 Hugging Face Hub 的 create_repo 函数，用于创建一个新的模型仓库
from huggingface_hub import create_repo

# 在 Hugging Face Hub 上创建一个新的模型仓库
create_repo("whatfa/miaoshouhuichun", token=HUGGINGFACE_TOKEN, exist_ok=True)

# 将模型和分词器上传到 Hugging Face Hub 上的仓库
model.push_to_hub_gguf("whatfa/miaoshouhuichun", tokenizer, token=HUGGINGFACE_TOKEN)

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 4.19 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [01:48<00:00,  3.38s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving whatfa/miaoshouhuichun/pytorch_model-00001-of-00004.bin...
Unsloth: Saving whatfa/miaoshouhuichun/pytorch_model-00002-of-00004.bin...
Unsloth: Saving whatfa/miaoshouhuichun/pytorch_model-00003-of-00004.bin...
Unsloth: Saving whatfa/miaoshouhuichun/pytorch_model-00004-of-00004.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at whatfa/miaoshouhuichun into q8_0 GGUF format.
The output location will be /content/whatfa/miaoshouhuichun/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: miaoshouhuichun
INFO:gguf.gguf_writer:

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q8_0.gguf:   0%|          | 0.00/8.54G [00:00<?, ?B/s]

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Saved GGUF to https://huggingface.co/whatfa/miaoshouhuichun
